In [1]:
# If you are running on `Binder`, then it is no need to set up the packages again

# UNCOMMENT THIS TO INSTALL THE REQUIRED LIBRARIES
# %pip install -r requirements.txt
%pip install ipykernel -q

Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy as np
import pandas as pd
import sklearn
import tensorflow as tf
import pymysql
import sshtunnel

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tensorflow.keras.models import load_model

print("Numpy version:", np.__version__)
print("Pandas version:", pd.__version__)
print("Tensorflow version:", tf.__version__)
print("Sklearn version:", sklearn.__version__)
print("Pymysql version:", pymysql.__version__)
print("Sshtunnel version:", sshtunnel.__version__)

import os
try:
    # This will work in a script
    SCRIPT_DIR = os.path.dirname(os.path.abspath(__file__))
except NameError:
    # __file__ is not defined in notebooks; fallback to current working directory
    SCRIPT_DIR = os.getcwd()

MODEL_PATH = os.path.join(os.path.dirname(SCRIPT_DIR), "model", "ann_gym_recommender.keras")
DATA_PATH = os.path.join(os.path.dirname(SCRIPT_DIR), "data", "megaGymDataset.csv")

Numpy version: 1.26.4
Pandas version: 2.2.3
Tensorflow version: 2.10.1
Sklearn version: 1.6.1
Pymysql version: 1.4.6
Sshtunnel version: 0.4.0


### Reading data from a SQL-SERVER

In [3]:
try:
    # Set up SSH tunnel
    server = sshtunnel.SSHTunnelForwarder(
        ('selene.hud.ac.uk', 22),
        ssh_username='workit',
        ssh_password='umbra(despair>Quartz218',
        remote_bind_address=('localhost', 3306)
    )
    server.start()
    
    # Defining the connection
    connection = pymysql.connect(host='localhost', port=server.local_bind_port, user='workit', password='umbra(despair>Quartz218', database='workit')

    # Load the dataset
    dataset = pd.read_sql_query("SELECT * FROM `megaGymDataset`", connection)

    server.stop()
except:
    print("Could not connect to Database, reading local file from `" + DATA_PATH + "` instead... ")
    dataset = pd.read_csv(DATA_PATH)

2025-05-12 03:58:55,362| ERROR   | Could not resolve IP address for selene.hud.ac.uk, aborting!


Could not connect to Database, reading local file from `d:\ThongLai\Documents\Code\Web\WorkIt.Gym-Recommender-Website\AI\data\megaGymDataset.csv` instead... 


In [4]:
dataset.head()

,ID,Title,Desc,Type,BodyPart,Equipment,Level,Rating
0,0,Partner plank band row,The partner plank band row is an abdominal exe...,Strength,Abdominals,Bands,Intermediate,0.0
1,1,Banded crunch isometric hold,The banded crunch isometric hold is an exercis...,Strength,Abdominals,Bands,Intermediate,4.5
2,2,FYR Banded Plank Jack,The banded plank jack is a variation on the pl...,Strength,Abdominals,Bands,Intermediate,4.5
3,3,Banded crunch,The banded crunch is an exercise targeting the...,Strength,Abdominals,Bands,Intermediate,4.5
4,4,Crunch,The crunch is a popular core exercise targetin...,Strength,Abdominals,Bands,Intermediate,4.5


### Predict from user input

In [5]:
def labelEncodersScaler(data):
    label_encoders = {}
    for feature in ["Type", "BodyPart", "Equipment", "Level"]:
        label_encoders[feature] = LabelEncoder()
        data[feature] = label_encoders[feature].fit_transform(data[feature])

    X = data[["Type", "BodyPart", "Equipment", "Level"]]
    y = data["ID"]
    
    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Standardize the features
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)

    return label_encoders, scaler

In [6]:
def recommend_exercises(user_input, model, data):
    label_encoders, scaler = labelEncodersScaler(data)
    
    user_input_encoded = [label_encoders[feature].transform([user_input[i]])[0] for i, feature in enumerate(["Type", "BodyPart", "Equipment", "Level"])]
    user_input_scaled = scaler.transform([user_input_encoded])
    
    predictions = model.predict(user_input_scaled)
    
    return data.iloc[np.squeeze(predictions.argsort())][::-1][:10].sort_values(["Rating"], ascending=False).index.to_list()

In [7]:
model = load_model(MODEL_PATH)
user_input = ["Strength", "Biceps", "Body Only", "Intermediate"]
IDs = recommend_exercises(user_input, model, dataset.copy())
dataset.iloc[IDs]

1/1 [==============================] - 1s 691ms/step


,ID,Title,Desc,Type,BodyPart,Equipment,Level,Rating
855,855,UN Pull-Up Supinated,Pull-up variation with a supinated (underhand)...,Strength,Biceps,Body Only,Intermediate,9.000000
854,854,Pull-up - Gethin Variation,The pull-up is a multi-joint bodyweight exerci...,Strength,Biceps,Body Only,Intermediate,9.000000
857,857,UN Pull-Up Neutral,"Pull-up variation with a neutral grip, targeti...",Strength,Biceps,Body Only,Intermediate,9.000000
853,853,TBS Chin-Up,The a chin-up is a variation of the pull-up ex...,Strength,Biceps,Body Only,Intermediate,9.000000
16,16,30 Barbell Floor Wiper,The barbell floor wiper is a core exercise in ...,Strength,Abdominals,Barbell,Intermediate,7.750000
911,911,HM Left Calf-Spike Sumo Pulse,Dynamic calf exercise involving pulsing moveme...,Strength,Calves,Body Only,Intermediate,6.870833
684,684,Side-To-Side Adductor Stretch,Stretching the adductor muscles by moving legs...,Strength,Adductors,Body Only,Intermediate,4.816667
695,695,UNS Foam Roll Adductor,Using a foam roller to massage and release ten...,Strength,Adductors,Body Only,Intermediate,4.816667
694,694,HM Left Leg Swing,Dynamic leg swing exercise targeting hip flexo...,Strength,Adductors,Body Only,Intermediate,4.816667
4,4,Crunch,The crunch is a popular core exercise targetin...,Strength,Abdominals,Bands,Intermediate,4.500000
